# MLP Classiifers DEF

  - 3. Syndigo Mapping MachineLearning

In [1]:
def unique_no_nan(x):
    return x.dropna().unique()

In [2]:
def constrcut_dic():
    key = unique_no_nan(syndigo_mapped['Level 1'])
    value = list( map(lambda x : re.sub('&|/', '', x), key))
    value = list( map(lambda x : re.sub(' +', '_', x), value))
    value = list(map(str.lower, value))
    #value = ['-'.join(['tfidf'+ str]), value]

    # print(f'key: {key}') 
    # print(f'value: {value}')
    return dict(zip(key, value)) 

In [3]:
def prepare_data_1():
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [4]:
def prepare_data_2(level_1):
    global   k_neighbors
    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]

    series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)
    k_neighbors = 5
    if min(series) < 10:
        k_neighbors = 3 
        drop_cut_off = 10
        subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  < drop_cut_off ].index.tolist())].index) 
    print (f'Min cnt of Level 2 for class 1  {level_1}  is {min(series)} and k_neighbors is {k_neighbors}')
      
    x_subset_w2v = np.array(list(subset_df.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))                                            
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset_w2v  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map

In [5]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [6]:
def proc_pred_lr(level_1):
    lr_w2v = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga')
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    
    print(f"Training - w2v + LR \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    lr_w2v.fit(X_train, y_train)
    print(' Display LR test  metrics')
    preds_test = lr_w2v.predict(X_test)
    preds_train = lr_w2v.predict(X_train)
    print(classification_report(y_test, preds_test,labels = lr_w2v.classes_, target_names = [id2_level_map[i] for i in lr_w2v.classes_]))
    return preds_train, preds_test, lr_w2v

In [7]:
def proc_pred_mlp(level_1):

     mlp = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
     #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
     print(f"Training - w2v + MLP \"{level_1}\"")
     warnings.filterwarnings("ignore", category=UserWarning)
     mlp.fit(X_train, y_train)
     print(' Display MLP test  metrics')
     preds_test_mlp = mlp.predict(X_test)
     preds_train_mlp = mlp.predict(X_train)
    
     print(classification_report(y_test, preds_test_mlp,labels = mlp.classes_, target_names = [id2_level_map[i] for i in mlp.classes_]))      
     return preds_train_mlp, preds_test_mlp, mlp

In [8]:
def get_smp_strgy(dic_y_train, adj_ratio): 
    """
        get_smp_strgy  :  get sampling_strategy for SMOTE
    """
    smp_strgy = {} 
    
    adj_cnt =  int(max(dic_y_train.values()) * adj_ratio)
    for key, values in dic_y_train.items():
        if values < adj_cnt:
           smp_strgy[key] = adj_cnt
        else: 
            smp_strgy [key] = values
            
    return smp_strgy

In [19]:
def proc_smote(level_1): 
    print(f' smote level_1 : {level_1}')
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strg, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
                                                                                             
    print('Before Resampling:\n',pd.Series(y_train).value_counts())
    print('After Resampling:\n',pd.Series(y_resampled).value_counts())
    mlp_smote = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    print(f"Training - w2v + MLP \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    mlp_smote.fit(X_resampled, y_resampled)
    
    print(' Display MLP + SMOTE test  metrics')
    preds_test_mlp_smote = mlp_smote.predict(X_test)
    preds_train_mlp_smote = mlp_smote.predict(X_resampled)
    print(classification_report(y_test, preds_test_mlp_smote,labels = mlp_smote.classes_, target_names = [id2_level_map[i] for i in mlp_smote.classes_]))
    return preds_train_mlp_smote, preds_test_mlp_smote, mlp_smote

In [51]:
def validation(y_test, preds, level_1, model):
     global l_metric, l_metrics_v     
     pgm = inspect.currentframe().f_code.co_name       
     print(f'{pgm} for model {model} with level_1 = {level_1}')
     accuracy = accuracy_score(y_test,preds)
     f1_score_micro    = f1_score(y_test, preds, average='micro')  
     f1_score_weighted = f1_score(y_test, preds, average="weighted")
     f1_score_macro    = f1_score(y_test, preds, average='macro')
     print (f"ccuracy Score = {accuracy},f1_score_micro = {f1_score_micro}, f1_score_macro = {f1_score_macro}, f1_score_weighted = {f1_score_weighted}")
     l_metric = l_metric +  [model, accuracy, f1_score_micro, f1_score_macro, f1_score_weighted ]
     l_metrics_v.append([level_1, model, accuracy, f1_score_micro, f1_score_macro, f1_score_weighted ])  
     return 

In [17]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import Word2Vec, KeyedVectors
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
import re, time, inspect, pickle
import logging

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
import re

# load  function of 'elapse_time'
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"
exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))

In [12]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [13]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()

syndigo_mapped.shape (259085, 24)
syndigo_mapped['Level 1'].value_counts() Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
NaN                                           1132
Not classified                                1018
Hardware                                       923
Arts & 

In [14]:
dic_level1= constrcut_dic()


In [15]:
stop

NameError: name 'stop' is not defined

In [52]:
start_time = time.time() 

### import word2vec model trained on entire PIMMART data
#model = KeyedVectors.load_word2vec_format('/dbfs/FileStore/tables/DATA_SCIENCE/w2vmodel_053123_PIM_ALL.bin', binary=True)
model = KeyedVectors.load_word2vec_format(path + 'w2vmodel_053123_PIM_ALL.bin', binary=True)                                        
model.most_similar('chips')
adj_ratio = 0.2
i =0
err_smote_cnt = 0
err_smp_cnt = 0
#N = 5 
dic_level1= constrcut_dic()
#dic_level1 = dict(list(dic_level1.items())[4: N])
dic_scure={}
l_metrics = []
l_metrics_v = []
l_no_mdl = []
for level_1, filenamee in dic_level1.items():    
    print(f'{i}. Process level 1 = {level_1}')    
    i += 1    
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
    
    if  len ( x_subset_w2v) < 10: 
        len_x_subset_w2v = len(x_subset_w2v)
        print(f'len(x_subset_w2v) = {len_x_subset_w2v}. Not  process len(x_subset_w2v) < 10')
        l_no_mdl.append([level_1, 'Not  process len of data  < 10']) 
        continue
    dic_y = dict(pd.Series(y_subset_w2v).value_counts())  
    if len(dic_y.items())  == 1:
           print(f'Level1 {level_1} only has one level2. Will skip SMOTE' )
           l_no_mdl.append([level_1, 'Only has one level2'])  
           continue
    l_metric = [level_1]
    X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.3, stratify=y_subset_w2v, random_state=42)
    preds_train_lr, preds_test_lr, lr_w2v = proc_pred_lr(level_1)
    validation(y_test, preds_test_lr, level_1, model ='LR')
    
    preds_train_mlp, preds_test_mlp, mlp = proc_pred_mlp(level_1)
    validation(y_test, preds_test_mlp, level_1, model ='MLP')  
    
    dic_y_train = dict(pd.Series(y_train).value_counts())
    smp_strg = get_smp_strgy(dic_y_train, adj_ratio)
    preds_train_mlp_smote, preds_test_mlp_smote, mlp_smote =proc_smote(level_1)
    validation(y_test, preds_test_mlp_smote, level_1, model ='MLP_SMOTE')
    l_metrics.append(l_metric)
    
end_time = time.time()   
desc = f'Elapse_time for evluating LR, MLP and MLP_SMOTE'
elapse_time (  start_time, end_time, desc)

0. Process level 1 = Food / Beverages
Min cnt of Level 2 for class 1  Food / Beverages  is 309 and k_neighbors is 5
Training - w2v + LR "Food / Beverages"
 Display LR test  metrics
                                                          precision    recall  f1-score   support

                                    Dairy & Egg Products       0.89      0.88      0.88      2705
                                                 Grocery       0.91      0.96      0.93     22459
                                               Beverages       0.95      0.94      0.95      6372
Meat / Poultry / Seafood / Meat Substitutes (Perishable)       0.69      0.66      0.67      1193
                                            Frozen Foods       0.86      0.81      0.84      4074
                                           Bakery / Deli       0.56      0.29      0.39      1600
                                     Baby & Toddler Food       0.88      0.89      0.88       487
                                  

Training - w2v + LR "Beer / Wine / Spirits"
 Display LR test  metrics
                                precision    recall  f1-score   support

                          Beer       0.95      0.95      0.95      1837
  Flavored Alcoholic Beverages       0.53      0.50      0.52       262
                          Wine       0.98      0.99      0.98      2134
                       Spirits       0.95      0.95      0.95      1067
               Ciders & Perrys       0.77      0.77      0.77        61
Alcoholic Beverage Mixed Packs       0.17      0.07      0.10        15
           Non-Alcoholic Beers       0.40      0.46      0.43        13
   Alcoholic Cordials & Syrups       0.00      0.00      0.00         4
       Alcohol Making Supplies       0.67      0.50      0.57         4

                      accuracy                           0.94      5397
                     macro avg       0.60      0.58      0.59      5397
                  weighted avg       0.93      0.94      0.94   

 Display MLP test  metrics
                             precision    recall  f1-score   support

             Air Fresheners       0.94      0.97      0.95       532
         Cleaning Chemicals       0.97      0.96      0.96      1345
             Paper Products       1.00      0.97      0.98       281
     General Cleaning Tools       0.86      0.91      0.89       285
     Storage & Organization       0.92      0.86      0.89       122
          Trash & Recycling       0.95      0.97      0.96       115
         Product Dispensers       0.45      0.36      0.40        14
        Carpet & Floor Care       0.00      0.00      0.00         6
Moisture & Mildew Absorbers       1.00      0.57      0.73         7
 Vacuum Parts & Accessories       0.75      0.75      0.75         8
      Vacuums & Air Dusters       0.50      0.50      0.50         6

                   accuracy                           0.95      2721
                  macro avg       0.76      0.71      0.73      2721
     

 Display MLP + SMOTE test  metrics
                             precision    recall  f1-score   support

               Pet Supplies       1.00      1.00      1.00      1889
Livestock & Equine Supplies       0.67      0.67      0.67         9

                   accuracy                           1.00      1898
                  macro avg       0.83      0.83      0.83      1898
               weighted avg       1.00      1.00      1.00      1898

validation for model MLP_SMOTE with level_1 = Livestock & Pet Supplies
ccuracy Score = 0.9968387776606955,f1_score_micro = 0.9968387776606955, f1_score_macro = 0.8325392623963296, f1_score_weighted = 0.9968387776606955
7. Process level 1 = Toys / Games / Hobbies
Min cnt of Level 2 for class 1  Toys / Games / Hobbies  is 1 and k_neighbors is 3
Training - w2v + LR "Toys / Games / Hobbies"
 Display LR test  metrics
                             precision    recall  f1-score   support

            Games & Puzzles       0.89      0.85      0.87    

Before Resampling:
 1    412
0    353
4     34
3     29
2     19
5     12
6      8
dtype: int64
After Resampling:
 1    412
0    353
4     82
5     82
2     82
3     82
6     82
dtype: int64
Training - w2v + MLP "Tobacco Products"
 Display MLP + SMOTE test  metrics
                                  precision    recall  f1-score   support

                      Cigarettes       0.97      0.94      0.95       152
                          Cigars       0.97      0.97      0.97       177
                 Chewing Tobacco       0.56      0.62      0.59         8
                           Snuff       0.92      0.92      0.92        13
                      Vaporizers       0.71      0.36      0.48        14
                     Raw Tobacco       0.50      0.80      0.62         5
Electronic Cigarette Accessories       0.10      0.33      0.15         3

                        accuracy                           0.92       372
                       macro avg       0.68      0.71      0.67   

...

Training - w2v + LR "Appliances"
 Display LR test  metrics
                     precision    recall  f1-score   support

   Small Appliances       0.98      1.00      0.99       196
Parts & Accessories       1.00      0.57      0.73         7

           accuracy                           0.99       203
          macro avg       0.99      0.79      0.86       203
       weighted avg       0.99      0.99      0.98       203

validation for model LR with level_1 = Appliances
ccuracy Score = 0.9852216748768473,f1_score_micro = 0.9852216748768473, f1_score_macro = 0.8598388952819334, f1_score_weighted = 0.9832625689456769
Training - w2v + MLP "Appliances"
 Display MLP test  metrics
                     precision    recall  f1-score   support

   Small Appliances       0.98      0.99      0.99       196
Parts & Accessories       0.80      0.57      0.67         7

           accuracy                           0.98       203
          macro avg       0.89      0.78      0.83       203
      

 Display MLP + SMOTE test  metrics
                              precision    recall  f1-score   support

Fluids, Chemicals, Compounds       0.98      0.98      0.98       172
    Electrical & Electronics       0.90      0.64      0.75        14
                 Accessories       0.82      0.79      0.81        29
                      Engine       0.67      1.00      0.80         4
                       Tools       0.50      0.56      0.53         9
 Chassis Interior & Exterior       0.50      0.60      0.55         5

                    accuracy                           0.91       233
                   macro avg       0.73      0.76      0.73       233
                weighted avg       0.92      0.91      0.91       233

validation for model MLP_SMOTE with level_1 = Automotive
ccuracy Score = 0.9141630901287554,f1_score_micro = 0.9141630901287554, f1_score_macro = 0.7347496706192359, f1_score_weighted = 0.9144968606523688
20. Process level 1 = Tools
Min cnt of Level 2 for class 

Before Resampling:
 3    206
1     76
4     74
2     72
5     39
6     28
0     22
7     15
9     10
8      8
dtype: int64
After Resampling:
 3    206
1     76
4     74
2     72
5     41
0     41
6     41
9     41
7     41
8     41
dtype: int64
Training - w2v + MLP "Arts & Crafts"
 Display MLP + SMOTE test  metrics
                              precision    recall  f1-score   support

         Pottery & Sculpting       0.57      0.89      0.70         9
Paper / Canvas / Wood Boards       0.63      0.88      0.73        33
          Painting & Drawing       0.78      0.68      0.72        31
               Drawing Tools       0.80      0.73      0.76        88
                  Craft Kits       0.46      0.50      0.48        32
                    Stickers       0.50      0.50      0.50        16
     Needlework & Toy Making       0.78      0.58      0.67        12
              Jewelry Making       0.00      0.00      0.00         7
         Paper & Card Making       1.00      0.75   

Min cnt of Level 2 for class 1  Flooring  is 1 and k_neighbors is 3
len(x_subset_w2v) = 0. Not  process len(x_subset_w2v) < 10
32. Process level 1 = Paints & Coatings
Min cnt of Level 2 for class 1  Paints & Coatings  is 1 and k_neighbors is 3
Level1 Paints & Coatings only has one level2. Will skip SMOTE
33. Process level 1 = Building Supplies
Min cnt of Level 2 for class 1  Building Supplies  is 1 and k_neighbors is 3
Level1 Building Supplies only has one level2. Will skip SMOTE
34. Process level 1 = Musical Instruments
Min cnt of Level 2 for class 1  Musical Instruments  is 1 and k_neighbors is 3
len(x_subset_w2v) = 0. Not  process len(x_subset_w2v) < 10
35. Process level 1 = Material Handling
Min cnt of Level 2 for class 1  Material Handling  is 1 and k_neighbors is 3
len(x_subset_w2v) = 0. Not  process len(x_subset_w2v) < 10
36. Process level 1 = Agricultural Equipment
Min cnt of Level 2 for class 1  Agricultural Equipment  is 1 and k_neighbors is 3
len(x_subset_w2v) = 0. Not  proc

(0, 3, 15)

In [ ]:
def save_excel_pred( excel_file, dict):
#  Write classification report to excel
#  save() is going to remove, use close() instead
   from pandas import ExcelWriter
   from pandas import ExcelFile
   writer = pd.ExcelWriter(excel_file)
   for key,  value in dict.items():
       df   =  value[0] 
       tab  =  value[1] 
       df.to_excel(writer,tab)
   writer.close()
   return

In [53]:
 l_metrics_v
    

[['Food / Beverages',
  'LR',
  0.8923197730381478,
  0.8923197730381478,
  0.7006422435326318,
  0.8852829648053735],
 ['Food / Beverages',
  'MLP',
  0.9124575713055373,
  0.9124575713055373,
  0.7442185880422753,
  0.9083384138645368],
 ['Food / Beverages',
  'MLP_SMOTE',
  0.8974365469375348,
  0.8974365469375348,
  0.7313464604068064,
  0.898724130982642],
 ['Health & Beauty',
  'LR',
  0.9165328193596745,
  0.9165328193596743,
  0.8861238142258948,
  0.9161534103013022],
 ['Health & Beauty',
  'MLP',
  0.925206124852768,
  0.925206124852768,
  0.8981819509030655,
  0.9245497986152366],
 ['Health & Beauty',
  'MLP_SMOTE',
  0.9219402505621587,
  0.9219402505621587,
  0.8963956151918708,
  0.9215118579647176],
 ['Beer / Wine / Spirits',
  'LR',
  0.9375579025384473,
  0.9375579025384473,
  0.5852076199022631,
  0.9361017788475984],
 ['Beer / Wine / Spirits',
  'MLP',
  0.9336668519547897,
  0.9336668519547897,
  0.5569099971359098,
  0.9326810568329434],
 ['Beer / Wine / Spirits',


In [42]:
lr_col =  ['lr_modell', 'lr_accuracy',  'lr_f1_score_micro', 'lr_f1_score_macro', 'lr_f1_score_weighted']
mlp_col =  ['mlp_modell', 'mlp_accuracy',  'mlp_f1_score_micro', 'mlp_f1_score_macro', 'mlp_f1_score_weighted']
smote_col =  ['smote_modell', 'smote_accuracy',  'smote_f1_score_micro', 'smote_f1_score_macro', 'smote_f1_score_weighted']
col = ['Level1'] +  lr_col + mlp_col + smote_col
df_scr = pd.DataFrame(l_metrics, columns = col)
#df_scr.to_excel(writer,'lr_mlp_smote_scores')  
df_scr.head()

,Level1,lr_modell,lr_accuracy,lr_f1_score_micro,lr_f1_score_macro,lr_f1_score_weighted,mlp_modell,mlp_accuracy,mlp_f1_score_micro,mlp_f1_score_macro,mlp_f1_score_weighted,smote_modell,smote_accuracy,smote_f1_score_micro,smote_f1_score_macro,smote_f1_score_weighted
0,Food / Beverages,LR,0.892320,0.892320,0.700424,0.885283,MLP,0.912888,0.912888,0.737576,0.909787,MLP_SMOTE,0.901109,0.901109,0.732746,0.901506
1,Health & Beauty,LR,0.916479,0.916479,0.886055,0.916104,MLP,0.925634,0.925634,0.900296,0.925478,MLP_SMOTE,0.926491,0.926491,0.899489,0.926266
2,Beer / Wine / Spirits,LR,0.937187,0.937187,0.583883,0.935753,MLP,0.932370,0.932370,0.540963,0.931881,MLP_SMOTE,0.929220,0.929220,0.570775,0.931615
3,Kitchen & Bathroom,LR,0.986717,0.986717,0.755494,0.986009,MLP,0.980392,0.980392,0.807220,0.979753,MLP_SMOTE,0.986085,0.986085,0.819206,0.985411
4,Cleaning & Janitorial,LR,0.950018,0.950018,0.774527,0.949543,MLP,0.944873,0.944873,0.746163,0.944149,MLP_SMOTE,0.944506,0.944506,0.776562,0.944640


In [44]:
excel_file = path + "lr_mlp_smote_scores_" + dte +'.xlsx'
df_scr.to_excel(excel_file, index=None)


In [ ]:
from datetime import date
dte = date.today().strftime('%m%d%y')
excel_file = path + "lr_mlp_smote_scores_"+dte
from pandas import ExcelWriter
from pandas import ExcelFile
writer = pd.ExcelWriter(excel_file)
lr_col =  ['lr_modell', 'lr_accuracy',  'lr_f1_score_micro', 'lr_f1_score_macro', 'lr_f1_score_weighted']
mlp_col =  ['mlp_modell', 'mlp_accuracy',  'mlp_f1_score_micro', 'mlp_f1_score_macro', 'mlp_f1_score_weighted']
smote_col =  ['smote_modell', 'smote_accuracy',  'smote_f1_score_micro', 'smote_f1_score_macro', 'smote_f1_score_weighted']
col = ['Level1'] +  lr_col + mlp_col + smote_col
df_scr = pd.DataFrame(l_metrics, columns = col)
df_scr.to_excel(writer,'lr_mlp_smote_scores')  
df_no_model = pd.DataFrame(l_no_mdl, columns = ['level1', 'reason'])       
writer.close()


In [41]:
df_scr.head() 

NameError: name 'df_scr' is not defined

In [56]:
from datetime import date
dte = date.today().strftime('%m%d%y')
excel_file = path + "lr_mlp_smote_scores_"+ dte + '.xlsx'
from pandas import ExcelWriter
from pandas import ExcelFile
writer = pd.ExcelWriter(excel_file)
lr_col =  ['lr_modell', 'lr_accuracy',  'lr_f1_score_micro', 'lr_f1_score_macro', 'lr_f1_score_weighted']
mlp_col =  ['mlp_modell', 'mlp_accuracy',  'mlp_f1_score_micro', 'mlp_f1_score_macro', 'mlp_f1_score_weighted']
smote_col =  ['smote_modell', 'smote_accuracy',  'smote_f1_score_micro', 'smote_f1_score_macro', 'smote_f1_score_weighted']
col = ['Level1'] +  lr_col + mlp_col + smote_col
df_scr = pd.DataFrame(l_metrics, columns = col)
df_scr.to_excel(writer,'lr_mlp_smote_scores')  

df_scr_v = pd.DataFrame(l_metrics_v, columns = ['level1', 'modell', 'accuracy',  'f1_score_micro', 'f1_score_macro', 'f1_score_weighted'])
df_scr_v.to_excel(writer,'lr_mlp_smote_scores_v') 
df_no_model = pd.DataFrame(l_no_mdl, columns = ['level1', 'reason'])  
df_no_model.to_excel(writer,'Level1 not build model')
writer.close()


## wip

In [ ]:
import logging
try:
       proc_smote()
except Exception as e:
       logging.error("Exception occurred", exc_info=True)
print("test")

In [ ]:
len(dic_level1 )

In [ ]:
syndigo_mapped.columns

In [ ]:
 k_neighbors

In [ ]:
subset_df['Level 2'].value_counts(ascending=True, dropna = False)

In [ ]:
syndigo_mapped[syndigo_mapped['Level 1']==level_1]['Level 2'].value_counts(ascending=True,  dropna = False)

In [ ]:

#syndigo_mapped['Level 1' ==  level_1]['Level 2'].value_counts(dropna = False) 
min_cnt = min(syndigo_mapped[syndigo_mapped['Level 1']==level_1]['Level 2'].value_counts(ascending=True))
k_neighbors = 5
if min_cnt < 3 :  k_neighbors  +  min_cnt  

In [ ]:
level_1 = 'Health & Beauty'.

In [ ]:
syndigo_mapped['Level 1' ==  level_1]['Level 2'].value_counts() 

In [ ]:
min(subset_df['Level 2'].value_counts(ascending=True))

In [ ]:
series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)

In [ ]:

if min(series) < 3:
    k_neighbors = 3
    subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  < k_neighbors].index.tolist())].index) 

In [ ]:
#type(X_train) # numpy.ndarray
len(X_train) #28

In [ ]:


dic_level1= constrcut_dic()

dic_level1 = dict(list(dic_level1.items())[4: 5])

for level_1, filenamee in dic_level1.items():
    i += 1
    print(f'{i}. Process level 1 = {level_1}')
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
    X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.2, stratify=y_subset_w2v, random_state=42)
    proc_predict(level_1)
    dic_y_train = dict(pd.Series(y_train).value_counts())
    smp_strgy = get_smp_strgy(dic_y_train, adj_ratio)
    proc_smote()


In [ ]:
unique, couints = np.unique(y_train, return_counts = True)
y_dic =dict(zip(unique, couints )) 
y_dic

In [ ]:
dic_level1= constrcut_dic()
dic_level1.items()

In [ ]:
 dic_level1, level1 = wrapper(dic_srt= 13, dic_end= 14)

In [ ]:
validation(y_test, preds_train_lr, level_1, model ='LR')

In [ ]:
len(y_test)

In [ ]:
validation(y_test, preds_test_mlp_smote, level_1, model ='MLP_SMOTE')

In [28]:
l_metrics

[['LR',
  'Food / Beverages',
  0.8923197730381478,
  0.8923197730381478,
  0.7003112965691767,
  0.8852831527670381,
  'MLP',
  'Food / Beverages',
  0.9145600081057804,
  0.9145600081057804,
  0.7464440824269701,
  0.9107834747286542,
  'MLP_SMOTE',
  'Food / Beverages',
  0.89996960332337,
  0.89996960332337,
  0.7387451908460708,
  0.8997469331575133],
 ['LR',
  'Health & Beauty',
  0.9165863582824714,
  0.9165863582824714,
  0.8862171135158375,
  0.91620317341407,
  'MLP',
  'Health & Beauty',
  0.9222614840989399,
  0.9222614840989399,
  0.8954301363092924,
  0.9220915107376432,
  'MLP_SMOTE',
  'Health & Beauty',
  0.925206124852768,
  0.925206124852768,
  0.8973617061662302,
  0.9247000292107682],
 ['LR',
  'Beer / Wine / Spirits',
  0.937372614415416,
  0.937372614415416,
  0.5850403543570878,
  0.9359552939856824,
  'MLP',
  'Beer / Wine / Spirits',
  0.936816750046322,
  0.936816750046322,
  0.560456373320723,
  0.9323558756230005,
  'MLP_SMOTE',
  'Beer / Wine / Spirits',
 

In [ ]:
    lr_w2v = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'lbfgs')
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    
    print(f"Training - w2v + LR \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    lr_w2v.fit(X_train, y_train)

In [32]:
l_metricx
l_no_mdlx

[['Plumbing & Water Service', 'Only has one level2'],
 ['Not classified', 'Only has one level2'],
 ['Books & Videos', 'Only has one level2'],
 ['Power Sports', 'Only has one level2'],
 ['Power Transmission & Motion Control', 'Not  process len of data  < 10'],
 ['Marine', 'Not  process len of data  < 10'],
 ['Flooring', 'Not  process len of data  < 10'],
 ['Paints & Coatings', 'Only has one level2'],
 ['Building Supplies', 'Only has one level2'],
 ['Musical Instruments', 'Not  process len of data  < 10'],
 ['Material Handling', 'Not  process len of data  < 10'],
 ['Agricultural Equipment', 'Not  process len of data  < 10']]

In [25]:
len(l_no_mdl) 

12